# Diseño de experimento
## Una variable a la vez

El enfoque de diseño de experimentos de una variable a la vez (One Variable At a Time, OVAT) es una metodología simple en la que se varía un solo factor o variable experimental mientras se mantienen constantes todos los demás factores. Este enfoque permite observar cómo cambios en esa única variable afectan el resultado del experimento. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/irvingvasquez/practicas_pytorch/blob/master/soluciones/06_doe_una_variable.ipynb)

Si ejecutas en COLAB debes copiar los archivos extra de este repositorio.

@juan1rving


In [2]:
# Cargamos paquetes necesarios

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

#helper was developed by Udacity under MIT license
import helper

### 1. Definición de variables y rango de valores:

Identificar los factores o variables experimentales clave que se desean estudiar. En el contexto de redes neuronales, estas variables pueden incluir la tasa de aprendizaje, el número de capas, el número de neuronas por capa, el tamaño del lote, entre otros.

Variables que tomaremos en cuenta:

- Tasa de aprendizaje (eta)
- Número de épocas (n_epocas)
- Tamaño de lote (batch_size)

Una vez definidas las variables independientes definimos un rango de valores posibles para cada variable.

> TODO: Define un rango de valores posibles para cada variable. Incluye el valor mínimo y el valor máximo. Se sugiere utilizar una lista de valores obtenida con una separación uniforme. Probar almenos 5 valores por variable.




In [ ]:
#TODO: Define rangos para los hiperparámetros

### 2. Configuración inicial:

Establecer una configuración inicial para la red neuronal con valores predeterminados para todos los hiperparámetros. 

> TODO: Define la configuración inicial. Se sugiere usar un diccionario para contener dicha configuración.


In [12]:

configuracion = {'eta': 0, 'epochs': 0, 'batch_size': 0}

### 3. Variación de una variable a la vez:

- Seleccionar la primera variable a estudiar (por ejemplo, la tasa de aprendizaje).
- Realizar una serie de experimentos donde se varía únicamente la tasa de aprendizaje, mientras se mantienen constantes todos los demás hiperparámetros.
- Registrar el rendimiento del modelo para cada valor de la tasa de aprendizaje.

> TODO: Modifica el código para que pueda aceptar la configuración deseada

In [3]:
# Definimos una transformación de los datos
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5))])
# Descargamos el conjunto de entrenamiento y cargamos mediante un dataLoader
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Descargamos el conjunto de validación
validationset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=64, shuffle=True)

In [5]:
class RedNeuronal(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p = 0.5):
        '''
        Construye una red de tamaño arbitrario.
        
        Parámetros:
        input_size: cantidad de elementos en la entrada
        output_size: cantidada de elementos en la salida 
        hidden_layers: cantidad de elementos por cada capa oculta
        drop_p: probabilidad de "tirar" (drop) una neurona [0,1] 
        '''
        # llamamos al constructor de la superclase
        super().__init__()
        
        # Agregamos la primera capa
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # agregamos cada una de las capas, zip empareja el número de entradas con las salidas
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        # agregamos la capa de salida final de la red
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        # Incluimos drop-out en la red
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Pase hacia adelante en la red, el regreso son las probabilidades en el dominio log '''
        
        # Hacemos un pase frontal en cada una de las capas ocultas, 
        # La funció de activación es un RELU combinado con dropout
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [7]:
# Create the network, define the criterion and optimizer
model = RedNeuronal(784, 10, [516, 256], drop_p=0.5)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Implementamos una función de evaluación
def validation(model, validationloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in validationloader:

        images.resize_(images.shape[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [ ]:
#hiperparámetro
epochs = 2
steps = 0
running_loss = 0
print_every = 40
for e in range(epochs):
    # Cambiamos a modo entrenamiento
    model.train()
    for images, labels in trainloader:
        steps += 1
        
        # Aplanar imágenes a un vector de 784 elementos
        images.resize_(images.size()[0], 784)
        
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        # Backprogamation
        loss.backward()
        # Optimización
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Cambiamos a modo de evaluación
            model.eval()
            
            # Apagamos los gradientes, reduce memoria y cálculos
            with torch.no_grad():
                test_loss, accuracy = validation(model, validationloader, criterion)
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Pérdida de entrenamiento: {:.3f}.. ".format(running_loss/print_every),
                  "Pérdida de validación: {:.3f}.. ".format(test_loss/len(validationloader)),
                  "Exactitud de validación: {:.3f}".format(accuracy/len(validationloader)))
            
            running_loss = 0
            
            # Make sure training is back on
            model.train()

### 4. Selección del mejor valor:

Analizar los resultados y seleccionar el valor de la tasa de aprendizaje que produce el mejor rendimiento del modelo.


### 5. Repetición para otras variables:

Proceder con la siguiente variable (por ejemplo, el número de capas) y repetir el proceso: variar solo esta variable mientras se mantienen constantes todos los demás hiperparámetros, utilizando el mejor valor encontrado para la tasa de aprendizaje. Continuar este proceso para cada variable en la lista.

> TODO: Escribe una tabla con el resultado de cada experimento. Las columnas deben ser: ID, Configuración, Exactitud obtenida.

## Conclusiones

¿Cual fue el mejor valor encontrado?
¿Cuantas ejecuciones se realizaron?
¿Que tiempo tomó realizar todos los experimentos?